In [1]:
import findspark
findspark.init("/usr/local/spark")

In [2]:
from pyspark.sql import SparkSession

In [3]:
# May take awhile locally
spark = SparkSession.builder.appName("LogisticReg").getOrCreate()

In [4]:
import os
os.getenv('PYTHON_SPARK')

'/home/pasharma/workspace/CodingChills/PySpark'

In [5]:
data_dir = os.path.join(os.getenv('PYTHON_SPARK'),'Data')

In [6]:
from pyspark.ml.classification import LogisticRegression

In [8]:
#follow similar apprach as of linear reg
# Load training data
training = spark.read.format("libsvm").load(os.path.join(data_dir,"sample_libsvm_data.txt"))
#Create reg model obj
lr = LogisticRegression()

# Fit the model
lrModel = lr.fit(training)
#summary
trainingSummary = lrModel.summary

In [9]:
trainingSummary.predictions.show(2)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [10]:
# May change soon!
from pyspark.mllib.evaluation import MulticlassMetrics

In [11]:
lrModel.evaluate(training)

In [12]:
# Usually would do this on a separate test set!
predictionAndLabels = lrModel.evaluate(training)

In [13]:
predictionAndLabels.predictions.show(2)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[19.8534775947478...|[0.99999999761359...|       0.0|
|  1.0|(692,[158,159,160...|[-20.377398194908...|[1.41321555111056...|       1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 2 rows



In [14]:
predictionAndLabels = predictionAndLabels.predictions.select('label','prediction')

In [15]:
predictionAndLabels.show(2)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  1.0|       1.0|
+-----+----------+
only showing top 2 rows



## Evaluators

Evaluators will be a very important part of our pipline when working with Machine Learning, let's see some basics for Logistic Regression, useful links:

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.BinaryClassificationEvaluator

https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.evaluation.MulticlassClassificationEvaluator

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

In [17]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

In [18]:
# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

In [19]:
acc = evaluator.evaluate(predictionAndLabels)

In [20]:
acc

1.0

## Woking Eg

In [21]:
data = spark.read.csv(os.path.join(data_dir, 'titanic.csv'),inferSchema=True,header=True)

In [22]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [23]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [24]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [25]:
my_final_data = my_cols.na.drop()

In [26]:
my_final_data.show(2)

+--------+------+------+----+-----+-----+-------+--------+
|Survived|Pclass|   Sex| Age|SibSp|Parch|   Fare|Embarked|
+--------+------+------+----+-----+-----+-------+--------+
|       0|     3|  male|22.0|    1|    0|   7.25|       S|
|       1|     1|female|38.0|    1|    0|71.2833|       C|
+--------+------+------+----+-----+-----+-------+--------+
only showing top 2 rows



### Working with Categorical Columns

Let's break this down into multiple steps to make it all clear.

In [27]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [28]:
gender_indexer = StringIndexer(inputCol='Sex',outputCol='SexIndex')
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

In [30]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [31]:
assembler = VectorAssembler(inputCols=['Pclass',
 'SexVec',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'EmbarkVec'],outputCol='features')

In [32]:
from pyspark.ml.classification import LogisticRegression

## Pipelines 

Let's see an example of how to use pipelines (we'll get a lot more practice with these later!)

In [33]:
from pyspark.ml import Pipeline

In [34]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

In [35]:
pipeline = Pipeline(stages=[gender_indexer,embark_indexer,
                           gender_encoder,embark_encoder,
                           assembler,log_reg_titanic])

In [36]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [37]:
fit_model = pipeline.fit(train_titanic_data)

In [38]:
results = fit_model.transform(test_titanic_data)

In [39]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [40]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [44]:
results.select('Survived','prediction').show(5)

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
+--------+----------+
only showing top 5 rows



In [42]:
AUC = my_eval.evaluate(results)

In [43]:
AUC

0.7706845238095237

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [45]:
data = spark.read.csv(os.path.join(data_dir,'customer_churn.csv')
                      ,inferSchema=True,
                     header=True)

In [46]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [47]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [48]:
from pyspark.ml.feature import VectorAssembler

In [49]:
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [50]:
output = assembler.transform(data)

In [51]:
final_data = output.select('features','churn')

In [52]:
train_churn,test_churn = final_data.randomSplit([0.7,0.3])

In [53]:
from pyspark.ml.classification import LogisticRegression

In [54]:
lr_churn = LogisticRegression(labelCol='churn')

In [55]:
fitted_churn_model = lr_churn.fit(train_churn)

In [56]:
training_sum = fitted_churn_model.summary

In [57]:
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                601|                601|
|   mean|0.17470881863560733|0.13643926788685523|
| stddev|0.38003415206895386| 0.3435403422047109|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [58]:
training_sum.predictions

DataFrame[features: vector, churn: double, rawPrediction: vector, probability: vector, prediction: double]

In [61]:
training_sum.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,0.0...|  0.0|[4.93558466119832...|[0.99286501564864...|       0.0|
|[26.0,8939.61,0.0...|  0.0|[6.53762735074329...|[0.99855417360805...|       0.0|
|[28.0,8670.98,0.0...|  0.0|[7.84160296590410...|[0.99960711591047...|       0.0|
|[28.0,9090.43,1.0...|  0.0|[1.76361976359248...|[0.85366243064091...|       0.0|
|[28.0,11245.38,0....|  0.0|[4.02038168615941...|[0.98237027127007...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



### Evaluate results

Let's evaluate the results on the data set we were given (using the test data)

In [62]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [63]:
pred_and_labels = fitted_churn_model.evaluate(test_churn)

In [65]:
pred_and_labels.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.91527152983088...|[0.99271966568265...|       0.0|
|[26.0,8787.39,1.0...|    1|[0.92220915294205...|[0.71549202191954...|       0.0|
|[27.0,8628.8,1.0,...|    0|[5.60020884042414...|[0.99631652660461...|       0.0|
|[28.0,11128.95,1....|    0|[4.36255128970097...|[0.98741458371601...|       0.0|
|[28.0,11204.23,0....|    0|[2.06277171843477...|[0.88723178242734...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [66]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')

In [67]:
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [68]:
auc

0.7156167979002626

In [69]:
## Predictions on new customers

In [70]:
final_lr_model = lr_churn.fit(final_data)

In [71]:
new_customers = spark.read.csv(os.path.join(data_dir,'new_customers.csv'),inferSchema=True,
                              header=True)

In [72]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [73]:
test_new_customers = assembler.transform(new_customers)

In [74]:
final_results = final_lr_model.transform(test_new_customers)

In [75]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

